## The Transformer architecture

In [18]:
import random
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.utils import text_dataset_from_directory
from tensorflow.keras.layers import TextVectorization, Bidirectional, LSTM, Dropout, Dense, Layer, Embedding
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.initializers import Constant
import numpy as np

In [2]:
batch_size = 32
train_ds = text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
validation_ds = text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test_ds = text_dataset_from_directory("aclImdb/test", batch_size=batch_size)

Found 70000 files belonging to 3 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Vectorizing the data**

In [3]:
def clean_text(text):
    # Convert to lowercase
    text = tf.strings.lower(text)
    
    # Remove non-UTF-8 characters (ignore decoding errors)
    text = tf.strings.regex_replace(text, r'[^\x00-\x7F]+', '')  
    
    # Keep only letters, numbers, spaces, and some punctuation
    text = tf.strings.regex_replace(text, r"[^a-zA-Z0-9\s.,!?']", "")
    
    return text

def preprocess_text(text, label):
    
    text = clean_text(text)  # Apply cleaning function
    
    return text, label

In [7]:
train = train_ds.map(preprocess_text)
validation = validation_ds.map(preprocess_text)
test = test_ds.map(preprocess_text)

In [21]:
sequence_length = 600
max_tokens = 20000
vectorizer = TextVectorization(max_tokens=max_tokens, output_mode="int", output_sequence_length=sequence_length)

In [22]:
train_text = train.map(lambda x, y: x)
vectorizer.adapt(train_text)

In [23]:
int_train_ds = train.map(lambda x, y: (vectorizer(x), y), num_parallel_calls=4)
int_val_ds = validation.map(lambda x, y: (vectorizer(x), y), num_parallel_calls=4)
int_test_ds = test.map(lambda x, y: (vectorizer(x), y), num_parallel_calls=4)

**Transformer encoder implemented as a subclassed `Layer`**

In [12]:
class TransformerEncoder(Layer):

    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = Sequential([Dense(dense_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()

    def call(self, inputs, mask=None):
        
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        
        config = super().get_config()
        
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim
        })
        
        return config

**Using the Transformer encoder for text classification**

In [24]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

In [25]:
inputs = Input(shape=(None,), dtype="int64")
x = Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_2 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 257 

**Training and evaluating the Transformer encoder based model**

In [ ]:
callbacks = [ModelCheckpoint("transformer_encoder.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
2188/2188 [==============================] - 97s 43ms/step - loss: -2917.3035 - accuracy: 0.1428 - val_loss: 6717.3608 - val_accuracy: 0.5000
Epoch 2/20
2188/2188 [==============================] - 92s 42ms/step - loss: -16081.4248 - accuracy: 0.1429 - val_loss: 22966.8750 - val_accuracy: 0.5000
Epoch 3/20
2188/2188 [==============================] - 92s 42ms/step - loss: -40259.9844 - accuracy: 0.1429 - val_loss: 48738.9648 - val_accuracy: 0.5000
Epoch 4/20
2188/2188 [==============================] - 91s 42ms/step - loss: -75901.8047 - accuracy: 0.1429 - val_loss: 84664.7266 - val_accuracy: 0.5000
Epoch 5/20
2188/2188 [==============================] - 92s 42ms/step - loss: -123643.4062 - accuracy: 0.1429 - val_loss: 131337.4062 - val_accuracy: 0.5000
Epoch 6/20
2188/2188 [==============================] - 92s 42ms/step - loss: -183854.9844 - accuracy: 0.1429 - val_loss: 188788.8125 - val_accuracy: 0.5000
Epoch 7/20
2188/2188 [==============================] - 93s 42ms/ste

In [ ]:
model = load_model("transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")